## Task 1 (Detection of cars on main street)

In [1]:
pip uninstall opencv-python-headless -y 

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install opencv-python --upgrade

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Installation of Required Libraries 
%pip install pyttsx3
%pip install numpy
%pip install opencv-python
%pip install pypiwin32

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.



### Detect cars on the 'Main Street'.

In [4]:
import cv2

# Create VideoCapture object & reading frames from input file
# Ensuring path to video is correct 
video = cv2.VideoCapture('Ex1_files/Traffic_Laramie_1.mp4')

# Check if opening video is successful
if not video.isOpened(): 
    print("Error opening file!")

# Get video properties
fps = video.get(cv2.CAP_PROP_FPS)
frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define video writer object
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Choose the codec (mp4v or avc1)
output_video = cv2.VideoWriter("Detected.mp4", fourcc, fps, (frame_width, frame_height))

# Initialise initial frame to 'None'
initial_frame = None

# Looping through function to continue reading frames
while True:
    check, frame = video.read()
    if check:
        # Smoothening (Noise Reduction & Grey conversion)
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        blur_frame = cv2.GaussianBlur(gray_frame, (25, 25), 0)

        # 1st image = baseline image
        if initial_frame is None:
            initial_frame = blur_frame
            continue

        # Finding difference between baseline & frame images
        delta_frame = cv2.absdiff(initial_frame, blur_frame)

        # The difference between the initial frame and the current frame (the delta_frame) is transformed into a binary image.
        # When the pixel value is greater than 18,
        # The pixel will be rendered the colour white, otherwise, it is rendered the colour black 
        # threshold_frame = cv2.threshold(delta_frame, 18, 255, cv2.THRESH_BINARY)[1]

        # The kernel to apply to the morphology
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        closing = cv2.morphologyEx(delta_frame, cv2.MORPH_CLOSE, kernel)
        opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel)
        dilation = cv2.dilate(opening, kernel)
        retvalbin = cv2.threshold(dilation, 12, 255, cv2.THRESH_BINARY)[1]

        # cv2.findContours() method allows us to identify the contours in the image
        (contours, _) = cv2.findContours(retvalbin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Gets height of picture frame
        height = 0
        if video.isOpened():
            height = video.get(cv2.CAP_PROP_FRAME_HEIGHT)

        # This loops over each contour
        # Paints them on the original image
        for c in contours:
            # contourArea() will filter out the small contours
            if cv2.contourArea(c) < 2500:
                continue
            (x, y, w, h) = cv2.boundingRect(c)
            # If a car on the main street is near the bottom half of the video frame, paint the contour over it
            if y > height / 2:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 1)

        # Save the processed frame to the output video file
        output_video.write(frame)

        # Display the frame
        cv2.imshow('Video', frame)

        # Terminates when 'C' key is pressed
        if cv2.waitKey(1) & 0xFF == ord("c"):
            break

    else:
        break

# Releasing video objects after the loop is exited
video.release()
output_video.release()

# Closing all windows
cv2.destroyAllWindows()
